In [1]:
import sys
sys.path.append("./CVAE")

In [2]:
!pip install pyro-ppl
from main import main
import argparse
import itertools
import pandas as pd
import shutil
import os
import glob
import os

In [ ]:
# Define the hyperparameter search space
lr = [1e-3, 1e-3]
num_epochs = [2, 50]
num_particl = [20, 10]
z = [500] #, 250 makla
hidd1 = [500]  # Around the current hidden_1=500, 400, 500, makla
hidd2 = [1000]  # Around the current hidden_2=500

# Iterate over all combinations of hyperparameters
for lr, num_ep, num_part, z, hid1, hid2 in itertools.product(lr, num_epochs, num_particl, z, hidd1, hidd2):
    # Set up arguments
    args = argparse.Namespace(
        num_quadrant_inputs=[1, 2, 3],
        num_epochs=num_ep,
        early_stop_patience=3,
        learning_rate=lr,
        cuda=True,
        num_images=10,
        num_samples=10,
        num_particles=num_part,
        dataset='cifar10',
        z_dim=z,
        hidden_1=hid1,
        hidden_2=hid2,
        random_mask=False,
        allow_baseline_reuse=False,
        use_conv=False,
        pretrained=True
    )

    # Log the current configuration
    print(f"Running with lr={lr}, num_epochs={num_ep}, num_particles={num_part}, z_dim={z}, hidden_1={hid1}, hidden_2={hid2}")

    # Run the main function
    try:
        main(args)
    except Exception as e:
        print(f"Error with configuration lr={lr}, num_epochs={num_ep}, num_particles={num_part}, z_dim={z}, hidden_1={hid1}, hidden_2={hid2}: {e}")
        continue

    # Save the results
    if os.path.exists("results.csv"):
        output_file = f"CVAE/tuning_results_cifar10_pretrained/csvs/results_lr_{lr}_epochs_{num_ep}_particles_{num_part}_z_{z}_h1_{hid1}_h2_{hid2}.csv"
        shutil.copy("results.csv", output_file)
        print(f"Results saved to {output_file}")
    else:
        print(f"No 'results.csv' file generated for lr={lr}, num_epochs={num_ep}, num_particles={num_part}, z_dim={z}, hidden_1={hid1}, hidden_2={hid2}.")

    # Backup important files before starting
    for q in range(1, 4):
        plot_file = f"cvae_plot_q{q}.png"
        if os.path.exists(plot_file):
            output_file = f"CVAE/tuning_results_cifar10_pretrained/images/cvae_plot_q{q}_lr_{lr}_epochs_{num_ep}_particles_{num_part}_z_{z}_h1_{hid1}_h2_{hid2}.png"
            shutil.copy(plot_file, output_file)
            print(f"Backup created for '{plot_file}' as '{output_file}'.")


Running with lr=0.001, num_epochs=2, num_particles=20, z_dim=500, hidden_1=500, hidden_2=1000
Device:  cuda:0
Training with 1 quadrant as input...
Files already downloaded and verified
Files already downloaded and verified


CVAE Epoch 1 val    : 100%|██████████| 79/79 [00:02<00:00, 39.31it/s, early_stop_count=0, loss=1474.36]


Files already downloaded and verified
Files already downloaded and verified


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-1.0..1.0].


Files already downloaded and verified
Files already downloaded and verified


Generating predictions:  71%|███████   | 223/313 [00:37<00:17,  5.18it/s]

In [ ]:
# Combine all results for analysis
result_files = glob.glob("CVAE/tuning_results_cifar10_pretrained/csvs/*.csv")
if result_files:
    combined_results = []

    for file in result_files:
        # Read each result file
        df = pd.read_csv(file)

        # Rename the first column if necessary
        if 'Unnamed: 0' in df.columns:
            df.rename(columns={'Unnamed: 0': 'Method'}, inplace=True)

        print(df)

        # Extract performance gaps and compute the aggregate gap
        # Extract performance gaps and compute the aggregate gap
        performance_gap_row = df.loc[df['Method'] == 'Performance gap']
        aggregate_gap = performance_gap_row[['1 quadrant', '2 quadrants', '3 quadrants']].sum(axis=1).values[0]

        # Store results with the file name and aggregate gap
        combined_results.append({
            "file": file,
            "aggregate_gap": aggregate_gap,
            "1 quadrant gap": performance_gap_row['1 quadrant'].values[0],
            "2 quadrants gap": performance_gap_row['2 quadrants'].values[0],
            "3 quadrants gap": performance_gap_row['3 quadrants'].values[0]
        })

    # Convert to a DataFrame for sorting and analysis
    combined_results_df = pd.DataFrame(combined_results)

    # Sort by aggregate gap in descending order (largest gap first)
    combined_results_df = combined_results_df.sort_values(by="aggregate_gap", ascending=False)

    # Save the combined results to a file
    combined_results_df.to_csv("CVAE/tuning_results_cifar10_pretrained/combined_results.csv", index=False)
    print("All results combined and saved to 'CVAE/tuning_results_cifar10_pretrained/combined_results.csv'.")

    # Display the best configuration
    print("Best configuration:")
    best_file = combined_results_df.iloc[0]['file']
    print(f"File: {best_file}, Aggregate Performance Gap: {combined_results_df.iloc[0]['aggregate_gap']}")

    # Optionally, load and display the details of the best result
    best_result = pd.read_csv(best_file)
    print("\nDetails of the best configuration:")
    print(best_result)
else:
    print("No results to combine.")
